In [1]:
# Replace with the name you want used in publications of this data set
name = "Jenna Schabdach"

In [2]:
from google.cloud import bigquery
import pandas as pd
from annotationHelperLib import *
from reportMarkingFunctions import *
from IPython.display import clear_output

In [3]:
# This variable is a list of words that will be highlighed in yellow in the reports. 
# To add a specific string to the list, enclose it in single or double quotes and follow it
# with a comma
# wordsToHighlight = ['oma', 'surg', 'pongiform', 'NF1', 'mass', 'persistent', 'unchanged', 'stable', 'lobe', 'prolongation', 'intracranial']
highlightYellow = ['CLINICAL', 'INDICATION', 'HISTORY', 'REASON', 'IMPRESSION', 'FINDINGS', 'BRAIN',
                   'limited', 'MRA', 'MRI', 'ORBIT']
highlightGray = ['The ventricles and sulci are normal in size shape and configuration',
                 'There are no areas of abnormal enhancement following contrast administration',
                 'The visualized major intracranial vessels appear patent',
                 'No intra or extraaxial fluid collections are identified',
                 'No extra-axial fluid collections are identified',
                 'There are no areas of abnormal parenchymal signal intensity',
                 'No gross focal parenchymal signal abnormality is seen',
                 'No definite abnormality is seen in the visualized portions of the orbits',
                 'There are no areas of abnormal parenchymal signal intensity on these sequences',
                 'There are no areas of abnormal parechymal signal intensity',
                 'There is no parenchymal lesion or abnormal fluid collection',
                 'There is no parenchymal lesion or abnormal extra-axial collection',
                 'There is no mass effect or midline shift',
                 'There is no mass effect midline shift',
                 'No mass mass effect or abnormal enhancement is ',
                 'There is no mass or mass effect seen',
                 'No areas of hemorrhage are identified', 
                 'There is no abnormal intra- or extra-axial fluid collections',
                 'There is no evidence of intracranial hemorrhage or infarct',
                 'The ventricles and sulci are normal in size and configuration', 
                 'No parenchymal lesions or abnormal extra-axial collections are identified',
                 'There is no signal abnormality identified in the visualized parenchyma',
                 'There is no midline shift mass effect abnormal enhancement or acute infarction',
                 'There is no evidence of mass mass effect or midline shift',
                 'There is no intracranial mass lesion mass effect or midline shift',
                 'No intracranial hemorrhage midline shift or mass effect is appreciated',
                 'There are no mass effect midline shift abnormal enhancement or acute infarction',
                 'No mass effect midline shift or acute infarction is present',
                 'There are no intracranial mass lesions mass effect or midline shift',
                 'There is no intracranial mass effect or midline shift',
                 'There is no mass effect midline shift abnormal enhancement or acute infarction',
                 'No mass effect midline shift acute infarction or intracranial hemorrhage is identified',
                 'No mass effect midline shift abnormal enhancement or acute infarction is present',
                 'There is no brain parenchymal signal abnormality mass effect pathologic enhancement or restricted diffusion',
                 'There are no brain mass lesions mass effect or midline shift',
                 'There are no intracranial mass lesions',
                 'There is no mass effect midline shift or acute infarction',
                 'There are no areas of signal abnormality or pathologic enhancement', 
                 'No extra-axial collection mass effect or shift',
                 'no mass effect or midline shift',
                 'No intracranial mass lesion mass effect or midline shift is noted', 
                 'There are no intra- or extra-axial fluid collections',
                 'No abnormal fluid collections are identified',
                 'No extra-axial fluid collections are seen',
                 'No intra- or extra-axial fluid collections are identified',
                 'No abnormal extra-axial fluid collections are identified',
                 'No extraaxial fluid collections are identified',
                 'The midline structures are within normal limits',
                 'There is no restricted diffusion',
                 'The ventricular system is normal in size shape and configuration',
                 'The cerebellar tonsils are above the level of the foramen magnum',
                 'The cerebellar tonsils are at the level of foramen magnum',
                 'The cerebellar tonsils are at the level of the foramen magnum',
                 'The cerebellar tonsils do not project below the foramen magnum',
                 'The cerebellar tonsils are present at the foramen magnum',
                 'The cerebellar tonsils are above the foramen magnum',
                 'The cerebellar tonsils terminate above the level of the foramen magnum',
                 'The cerebellar tonsils terminate at the level of the foramen magnum',
                 'The cerebellar tonsils terminate at the foramen magnum', 
                 'The cerebellar tonsils are seen at the craniocervical junction',
                 'erebellar tonsils terminate above the foramen magnum',
                 'The cerebellar tonsils are at the foramen magnum',
                 'The cerebellar tonsils are appropriately positioned',
                 'yelination is appropriate for age',
                 'he myelination pattern is appropriate',
                 'Myelination is within normal limits for age',
                 'The myelination is within range for age',
                 'Myelination appears appropriate for age',
                 'The major intracranial vessels appear patent',
                 'The hippocampi are symmetric and normal in size signal and internal architecture',
                 'No parenchymal lesions or abnormal extraaxial collections are identified', 
                 'No parenchymal lesions or abnormal extra-axial collections are identified',
                 'There is no mass effect midline shift abnormal enhancement or acute infarction',
                 'There are no areas of abnormal enhancement following gadolinium administration',
                 'The visualized major intracranial flow voids are present',
                 'There is mild mucosal thickening',
                 'mucus retention cyst',
                 'mucous retention cyst',
                 'o midline shift mass effect or acute infarction',
                 'The major intracranial arterial structures demonstrate normal flow-related enhancement',
                 'The major intracranial flow voids are unremarkable', 
                 'The visualized portions of the orbits are grossly unremarkable',
                 'There is no appreciable alteration in course or caliber', 
                 'No areas of aneurysmal outpouching are appreciated',
                 'No parenchymal lesion or abnormal fluid collection is identified',
                 'The hippocampi are symmetric in size and of normal signal intensity',
                 'The middle ear cavities mastoid air cells and paranasal sinuses are clear',
                 'The visualized portions of the orbits and mastoid air cells are grossly unremarkable',
                 'It should be noted that MRA may be insensitive for small (<3-4mm) aneurysms',
                 'There are no intraparenchymal signal abnormalities seen', 
                 'The ventricles are symmetric and normal in size', 
                 'No focal lesions are identified',
                 'parenchyma demonstrates no signal abnormalities',
                 'The brain appears normal in morphology',
                 'The posterior fossa structures are normal', 
                 'No intra-axial fluid collections are identified',
                 'The major intracranial vessels demonstrate normal flow',
                 'o areas of abnormal enhancement following gadolinium administration',
                'cerebellar tonsils are rounded appearing and at the level of the foramen magnum']
highlightRed = ['artifact', 'rthodontic', 'istory of', 'motion', 'mass', 'cyst',
                'hypoplasia', 'signal abnormality', 'significant signal dropout',
                'hyperintens', 'prolongation', 'spongiform dysplasia',
                'volume loss']
highlightGreen = ['nremarkable brain', 'ormal brain',
                  'nremarkable', 'variant', 'variation', 
                  'normal anatomic variance', 'ncidental',
                  'within normal limit', 'developmental in etiology',
                  'doubtful significance', 'doubtful clinical significance',
                  'no clinical significance', 'no significance',
                  'without signal abnormality', 'o significant intracranial abnormality',
                  'o structural abnormality']
phrasesIndicatingContrast = ['MRI WITHOUT AND WITH CONTRAST', 'following intravenous contrast',
                            'after the injection of gadolinium', 'post gadolinium injection']

toHighlight = {'gray': highlightGray, 
               'green': highlightGreen,
               'yellow': highlightYellow,
               'red': highlightRed }

In [4]:
welcomeUser(name)

Welcome back, Jenna Schabdach
...
You currently have 68 ungraded reports to work on.


In [5]:

for i in range(25):
    clear_output()
    print(i)
    print()
    markOneReportSQL(name, toHighlight)


24

BRAIN MRI WITHOUT CONTRAST: CLINICAL INDICATION: Large intracranial arachnoid CYST. Follow-up evaluation for interval change. TECHNIQUE: Sagittal 3D T1 gradient echo with axial reformations axial and coronal TSE T2 axial and coronal FLAIR axial SWI arterial spin labeled perfusion imaging and axial diffusion imaging were performed on a 3.0 Tesla system. COMPARISON: Prior MRI dated January 24 2020 FINDINGS: Redemonstrated is a large arachnoid CYST filling the entire left middle cranial fossa and extending into the left frontal and temporal regions. It causes mild MASS effect and distortion of the underlying brain and 7 mm rightward shift of the midline structures unchanged. There is prominent anterior medial component with extension slightly across the midline unchanged. There is mild inferior displacement of the left tentorial leaflet and mild rightward displacement of the medial temporal lobe and the left cerebral peduncle unchanged. Mild thinning and smooth scalloping of the overl

Assign a CLIP rating to this report (0 do not use/1 maybe use/2 definitely use):  0



Grade saved. Run the cell again to grade another report.


In [ ]:
# getMoreReportsToGrade(name)